## Geospatial Analysis of Income and Business Locations Using Folium Maps

In [60]:
#Setup Notebook
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

In [61]:
# import CSV

#use 'SBAnational.csv' for full table

SBA_Loan = pd.read_csv('SBAnational.csv', low_memory=False)

### Import CSVs for state income and state abbrevations

In [62]:
state_income = pd.read_csv('state_income_clean.csv', low_memory=False)
state_code = pd.read_csv('state_codes.csv', low_memory=False)

In [63]:
state_code.head()

,Name,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


We need to be able to process the income dataframes and link them to the appropriate codes so that we can map them in the existing SBA Dataframe

In [64]:
#process state income and codes and add to existing SBA_Loans DF

def process_state_income(income_DF, states_DF,):
    income_DF = income_DF.rename(columns = {'State' : 'Name'})
    states_income = pd.merge(income_DF, states_DF, on = 'Name').rename(
        columns = {'Code' : 'State', 
                   'Date' : 'ApprovalFY', 
                   'Income' : 'Average_State_Income'
                  }).drop('Name', 1)
    states_income['ApprovalFY'] = states_income['ApprovalFY'].astype(str)
    
    return states_income
    
   

In [66]:
state_income_code = process_state_income(state_income, state_code)

In [67]:
state_income_code

,ApprovalFY,Average_State_Income,Factored Mean Income,State
0,2018,49936,0.790389,AL
1,2017,51113,0.832839,AL
2,2016,47221,0.799827,AL
3,2015,44509,0.787547,AL
4,2014,42278,0.787931,AL
...,...,...,...,...
1745,1988,26419,0.970395,WY
1746,1987,27590,1.058670,WY
1747,1986,23559,0.946259,WY
1748,1985,22081,0.934923,WY


The state_income_code table includes 'ApprovalFY' and 'State' which allow it to be mapped to the original SBA dataframe. The Factored Mean Income is the factor of the US average for the state.

### Total Average Income by Year (US)

We now want to calculate the average income by year in the US. We apply the groupby function for 'Approval Year' and find the mean of 'Average_State_Income'. 

In [9]:
year_average = state_income_code.groupby(['ApprovalFY']).mean().drop(columns = 'Factored Mean Income')

In [10]:
year_average.head()

,Average_State_Income
ApprovalFY,
1984,22441.04
1985,23215.26
1986,24333.88
1987,25617.94
1988,26835.22


### Total Average Income by State

We also want to investigate what the average income by state is. We apply the groupby function for the 'State' and find the mean of Average_State_Income. 

In [11]:
state_average = state_income_code.groupby(['State']).mean().sort_values(by = 'Average_State_Income',
                                                                            ascending=False)

state_average.head()

,Average_State_Income,Factored Mean Income
State,,
MD,53736.085714,1.291071
AK,52664.800000,1.284984
CT,52585.257143,1.274937
NJ,52420.857143,1.273195
NH,52146.800000,1.247873


### Map 'ApprovalFY' and 'State' to average state income 

We now want to create a function to merge the state income to the SBA dataframe. We merge on two columns: 'State' and 'ApprovalFY'. We add two new columns: 'Average_State_Income' and 'Factored Mean Income'

In [12]:
#function to integrate main df to include two columns 'Average_State_Income' and "Factored Mean Income"

def state_income(SBA_df, income_df):
    #where 'SBA_df' is the loan dataframe and 'income_df' is the state income df
    #both dfs should have compatible dtypes
    
    result = pd.merge(SBA_df, income_df, on=["State", "ApprovalFY"])
    
    return result

In [13]:
merged_df = state_income(SBA_Loan, state_income_code)

In [14]:
merged_df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Average_State_Income,Factored Mean Income
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",38889,1.050912
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00",38889,1.050912
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00",38889,1.050912
3,1001034000,CD WAREHOUSE,GREENWOOD,IN,46142,"PNC BANK, NATIONAL ASSOCIATION",IN,451220,28-Feb-97,1997,...,NaN,31-Aug-97,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",38889,1.050912
4,1001054006,"DAYS INN OF SOUTH BEND, L.P.",SOUTH BEND,IN,46637,"READYCAP LENDING, LLC",CA,0,3-Mar-97,1997,...,27-Sep-13,31-May-97,"$2,000,000.00",$0.00,CHGOFF,"$1,311,939.00","$2,000,000.00","$750,000.00",38889,1.050912


## Adding Inflation and Examining Year 2008 

In [15]:
income_SBA = merged_df
income_SBA


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Average_State_Income,Factored Mean Income
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",38889,1.050912
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00",38889,1.050912
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00",38889,1.050912
3,1001034000,CD WAREHOUSE,GREENWOOD,IN,46142,"PNC BANK, NATIONAL ASSOCIATION",IN,451220,28-Feb-97,1997,...,NaN,31-Aug-97,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",38889,1.050912
4,1001054006,"DAYS INN OF SOUTH BEND, L.P.",SOUTH BEND,IN,46637,"READYCAP LENDING, LLC",CA,0,3-Mar-97,1997,...,27-Sep-13,31-May-97,"$2,000,000.00",$0.00,CHGOFF,"$1,311,939.00","$2,000,000.00","$750,000.00",38889,1.050912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893030,9591593003,THE NEIGHBORHOOD CLEANERS,GREENWOOD,SC,29649,CERTIFIED DEVEL CORP OF SO. CA,SC,812320,28-Sep-96,1996,...,NaN,13-Aug-97,"$213,000.00",$0.00,P I F,$0.00,"$213,000.00","$213,000.00",34665,0.976699
893031,9591753003,WINGATE INN,COLUMBIA,SC,29223,CERTIFIED DEVEL CORP OF SO. CA,SC,0,28-Sep-96,1996,...,NaN,15-Mar-00,"$868,000.00",$0.00,P I F,$0.00,"$1,000,000.00","$1,000,000.00",34665,0.976699
893032,9591893010,RHETT HOUSE INN,BEAUFORT,SC,29902,CERTIFIED DEVEL CORP OF SO. CA,SC,0,28-Sep-96,1996,...,NaN,12-Nov-97,"$259,000.00",$0.00,P I F,$0.00,"$259,000.00","$259,000.00",34665,0.976699
893033,9591913003,"MERK PROPERTIES, INC.",COLUMBIA,SC,29223,CERTIFIED DEVEL CORP OF SO. CA,SC,0,28-Sep-96,1996,...,NaN,17-Dec-97,"$650,000.00",$0.00,P I F,$0.00,"$650,000.00","$650,000.00",34665,0.976699


We want to create two functions to further clean the data. We want to divide GrAppv and SBA_Appv in order to calculate how much of the GrAppv (Gross Loan Amount) is covered by the SBA (SBA_Appv). We have to remove the '$' signs. For the inflation, we have to remove the '%' signs

In [20]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return x.replace('$', '').replace(',', '')
    return x

def clean_pc(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return x.replace('%', '').replace(',', '')
    return x

In [21]:
income_SBA['GrAppv'] = income_SBA['GrAppv'].apply(clean_currency).astype('float')
income_SBA['SBA_Appv'] = income_SBA['SBA_Appv'].apply(clean_currency).astype('float')
income_SBA['Average_State_Income'] = income_SBA['Average_State_Income'].apply(clean_currency).astype('float')
income_SBA['Factored Mean Income'] = income_SBA['Factored Mean Income'].apply(clean_currency).astype('float')
income_SBA['Factored Mean Income'] = income_SBA['Factored Mean Income'].apply(clean_currency).astype('float')

income_SBA['SBA_backed_portion'] = income_SBA['SBA_Appv']/income_SBA['GrAppv']
income_SBA.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Average_State_Income,Factored Mean Income,SBA_backed_portion
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,60000.0,48000.0,38889.0,1.050912,0.800
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,31-May-97,"$40,000.00",$0.00,P I F,$0.00,40000.0,32000.0,38889.0,1.050912,0.800
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,287000.0,215250.0,38889.0,1.050912,0.750
3,1001034000,CD WAREHOUSE,GREENWOOD,IN,46142,"PNC BANK, NATIONAL ASSOCIATION",IN,451220,28-Feb-97,1997,...,31-Aug-97,"$60,000.00",$0.00,P I F,$0.00,60000.0,48000.0,38889.0,1.050912,0.800
4,1001054006,"DAYS INN OF SOUTH BEND, L.P.",SOUTH BEND,IN,46637,"READYCAP LENDING, LLC",CA,0,3-Mar-97,1997,...,31-May-97,"$2,000,000.00",$0.00,CHGOFF,"$1,311,939.00",2000000.0,750000.0,38889.0,1.050912,0.375


Let's extract the data for 2008 only since there are so many businesses. This will give us a good snapshot of loans, defaults, business, SBA-backed loan percentage for one year.

In [23]:
income_SBA['ApprovalFY'] = pd.to_datetime(income_SBA['ApprovalFY'], format = '%Y')

In [24]:
income_SBA_2008 = income_SBA[income_SBA['ApprovalFY'].dt.year == 2008]

In [25]:
income_SBA_2008

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Average_State_Income,Factored Mean Income,SBA_backed_portion
473124,2734655007,Jerome A. Cox and Cheryl L. Co,CASSVILLE,MO,65625,LIBERTY BANK,MO,722110,1-Oct-07,2008-01-01,...,30-Nov-07,"$127,918.00",$0.00,CHGOFF,"$48,799.00",130000.0,110500.0,46038.0,0.915214,0.85
473125,2734675002,"Neal Group Construction, LLC,",Joplin,MO,64801,LIBERTY BANK,MO,236220,1-Oct-07,2008-01-01,...,31-Dec-07,"$150,000.00",$0.00,P I F,$0.00,150000.0,127500.0,46038.0,0.915214,0.85
473126,2735315001,Michael L. Lane dba L & L Prec,Richland,MO,65556,COMMERCE BANK,MO,332710,1-Oct-07,2008-01-01,...,31-Oct-07,"$32,807.00",$0.00,P I F,$0.00,20000.0,10000.0,46038.0,0.915214,0.50
473127,2735545000,Serenity Covenat Llc,SAINT LOUIS,MO,63136,BBCN BANK,CA,812210,1-Oct-07,2008-01-01,...,31-Oct-07,"$10,000.00",$0.00,P I F,$0.00,10000.0,8500.0,46038.0,0.915214,0.85
473128,2735765007,"Professional Mortgage Group, I",Columbia,MO,65201,THE BANK OF MISSOURI,MO,522310,1-Oct-07,2008-01-01,...,31-Oct-07,"$16,388.00",$0.00,CHGOFF,"$9,399.00",10000.0,5000.0,46038.0,0.915214,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513289,3373166007,"SunnySide Laundromat, Carwash,",LENNOX,SD,57039,MINNWEST BANK,SD,811192,5-Aug-08,2008-01-01,...,31-Aug-08,"$72,000.00",$0.00,P I F,$0.00,72000.0,36000.0,51600.0,1.025784,0.50
513290,3398406003,CAPITOL INN & SUITES,PIERRE,SD,57501,KLEINBANK,MN,721110,22-Aug-08,2008-01-01,...,30-Sep-08,"$400,000.00",$0.00,P I F,$0.00,400000.0,300000.0,51600.0,1.025784,0.75
513291,3421726004,ROSS K ANDERSON,FLANDREAU,SD,57028,FIRST SAVINGS BANK,SD,484121,11-Sep-08,2008-01-01,...,28-Feb-09,"$20,243.00",$0.00,P I F,$0.00,10000.0,5000.0,51600.0,1.025784,0.50
513292,3425946008,TAN WORLD OF RAPID CITY INC,RAPID CITY,SD,57702,UNITED BANK OF IOWA,IA,812199,16-Sep-08,2008-01-01,...,28-Feb-09,"$372,000.00",$0.00,P I F,$0.00,372000.0,279000.0,51600.0,1.025784,0.75


In [26]:
income_SBA_2008['Default'] = np.where(income_SBA_2008['MIS_Status'] == 'CHGOFF', 1, 0)

In [27]:
income_SBA_2008['Default'].value_counts()

0    23253
1    16216
Name: Default, dtype: int64

In [28]:
#Get Average SBA_backed_portion by state

grouped_by_state = income_SBA_2008.groupby(['State']).agg('mean').reset_index()

# Folium and Interactive Maps
   

In [30]:
#!pip install folium
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


In [31]:
# initialize the map and store it in a m object
USmap = folium.Map(location=[40, -95], zoom_start=4, control_scale= True)

USmap

In [32]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data")
state_geo = f"{url}/us-states.json"


In [33]:
folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_average,
    columns=[state_average.index, "Average_State_Income"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Average State Income (present USD)",
).add_to(USmap)

folium.LayerControl().add_to(USmap)

USmap

In [34]:
import json
import utils

In [35]:
url_zip = (
    "https://raw.githubusercontent.com/millbj92/US-Zip-Codes-JSON/master")
zip_geo = f"{url_zip}/USCities.json"

In [36]:
zip_json = json.load(open(
    'UScities.json', encoding = 'utf8'))

#Print answer
utils.json_print(zip_json[0:5])

[
    {
        "city": "Holtsville",
        "county": "Suffolk",
        "latitude": 40.922326,
        "longitude": -72.637078,
        "state": "NY",
        "zip_code": 501
    },
    {
        "city": "Holtsville",
        "county": "Suffolk",
        "latitude": 40.922326,
        "longitude": -72.637078,
        "state": "NY",
        "zip_code": 544
    },
    {
        "city": "Adjuntas",
        "county": "Adjuntas",
        "latitude": 18.165273,
        "longitude": -66.722583,
        "state": "PR",
        "zip_code": 601
    },
    {
        "city": "Aguada",
        "county": "Aguada",
        "latitude": 18.393103,
        "longitude": -67.180953,
        "state": "PR",
        "zip_code": 602
    },
    {
        "city": "Aguadilla",
        "county": "Aguadilla",
        "latitude": 18.455913,
        "longitude": -67.14578,
        "state": "PR",
        "zip_code": 603
    }
]


In [37]:
# Convert dictionary into DataFrame
zipcodes = pd.DataFrame(zip_json)
zipcodes = zipcodes.rename(columns = {'zip_code' : 'Zip'}).drop(['city', 'state', 'county'], axis =1)

# Print answer
zipcodes

,Zip,latitude,longitude
0,501,40.922326,-72.637078
1,544,40.922326,-72.637078
2,601,18.165273,-66.722583
3,602,18.393103,-67.180953
4,603,18.455913,-67.14578
...,...,...,...
42736,99926,55.094325,-131.566827
42737,99927,55.517921,-132.003244
42738,99928,55.395359,-131.67537
42739,99929,56.449893,-132.364407


In [38]:
def SBA_geo(SBA_df, zipcodes_df):
    
    result = pd.merge(SBA_df, zipcodes_df, on=["Zip"])
    
    return result

In [39]:
SBA_geo = SBA_geo(income_SBA_2008, zipcodes)

In [40]:
SBA_geo

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Average_State_Income,Factored Mean Income,SBA_backed_portion,Default,latitude,longitude
0,2734655007,Jerome A. Cox and Cheryl L. Co,CASSVILLE,MO,65625,LIBERTY BANK,MO,722110,1-Oct-07,2008-01-01,...,CHGOFF,"$48,799.00",130000.0,110500.0,46038.0,0.915214,0.85,1,36.683825,-93.875744
1,2734675002,"Neal Group Construction, LLC,",Joplin,MO,64801,LIBERTY BANK,MO,236220,1-Oct-07,2008-01-01,...,P I F,$0.00,150000.0,127500.0,46038.0,0.915214,0.85,0,37.113343,-94.502663
2,2767355007,Dairan's Towing dba Larry's 24,Joplin,MO,64801,LIBERTY BANK,MO,488410,19-Oct-07,2008-01-01,...,P I F,$0.00,55000.0,46750.0,46038.0,0.915214,0.85,0,37.113343,-94.502663
3,2779965010,Karen S. Lawson d/b/a Roy's Co,Joplin,MO,64801,LIBERTY BANK,MO,532220,26-Oct-07,2008-01-01,...,P I F,$0.00,125000.0,106250.0,46038.0,0.915214,0.85,0,37.113343,-94.502663
4,2962155001,"J & S Brown, Inc, a Missouri C",Joplin,MO,64801,LIBERTY BANK,MO,238310,20-Feb-08,2008-01-01,...,CHGOFF,"$149,154.00",150000.0,127500.0,46038.0,0.915214,0.85,1,37.113343,-94.502663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39079,3228116007,MOOODY COUNTY DAIRY LIMITED PA,GARRETSON,SD,57030,PRAIRIELAND ECONOMIC DEVEL COR,MN,112120,17-Apr-08,2008-01-01,...,P I F,$0.00,2000000.0,2000000.0,51600.0,1.025784,1.00,0,43.747639,-96.596379
39080,3264186003,TERRY WHILE,MOBRIDGE,SD,57601,GREAT WESTERN BANK,SD,722211,27-May-08,2008-01-01,...,CHGOFF,"$75,567.00",90000.0,76500.0,51600.0,1.025784,0.85,1,45.550822,-100.330283
39081,3287846003,ML & K INC,TRENT,SD,57065,FIRST SAVINGS BANK,SD,811111,19-Jun-08,2008-01-01,...,P I F,$0.00,15000.0,7500.0,51600.0,1.025784,0.50,0,43.907616,-96.653301
39082,3373166007,"SunnySide Laundromat, Carwash,",LENNOX,SD,57039,MINNWEST BANK,SD,811192,5-Aug-08,2008-01-01,...,P I F,$0.00,72000.0,36000.0,51600.0,1.025784,0.50,0,43.298098,-96.856021


In [41]:
SBA_geo['latitude'] = pd.to_numeric(SBA_geo['latitude'], errors = 'coerce')
SBA_geo['longitude'] = pd.to_numeric(SBA_geo['longitude'], errors = 'coerce')

In [42]:
import geopandas as gpd

In [43]:
SBA_gdf = gpd.GeoDataFrame(SBA_geo, geometry = gpd.points_from_xy(
    SBA_geo.longitude, SBA_geo.latitude))


In [44]:
SBA_gdf = SBA_gdf.dropna(axis = 0, subset = ['latitude'])

In [45]:
SBA_gdf['Default'].value_counts()

0    22939
1    15974
Name: Default, dtype: int64

In [46]:
SBA_gdf['geometry'].isna().sum()

0

### Create Heat map of business loan locations

See attached Folium html file for final map

In [54]:
USmap2 = folium.Map(location=[40, -95], 
                    zoom_start=5, control_scale= True)

In [ ]:
# Add points to the map

featuregroup1 = folium.FeatureGroup(name = 
                                    "Loan Default Businesses",
                                  show = False)

mc = MarkerCluster(name = "Loan Default Businesses")
for idx, row in SBA_gdf.iterrows():
    if row.Default == 1:
        mc.add_child(Marker([row.geometry.y, 
                             row.geometry.x],
                     icon = folium.Icon(
                         icon = 'dot', 
                         color = 'red'))).add_to(USmap)

featuregroup1.add_child(mc)
USmap2.add_child(featuregroup1)
    
# Display map
USmap2


In [ ]:
# Add points to the map

featuregroup2 = folium.FeatureGroup(name = 
                                    "Approved Loan Businesses",
                                  show = False)

mc2 = MarkerCluster(name = "Approved Loan Businesses")
for idx, row in SBA_gdf.iterrows():
    if row.Default == 0:
        mc2.add_child(Marker([row.geometry.y, 
                              row.geometry.x], 
                            icon = folium.Icon(
                                icon = 'dot', color = 'green'))).add_to(USmap)

featuregroup2.add_child(mc2)
USmap2.add_child(featuregroup2)
    
# Display map
USmap2


In [ ]:
featuregroup3 = folium.FeatureGroup(name = "Average State Income (USD)",
                                  show = True, overlay = True)

cp_income = folium.Choropleth(
    geo_data=state_geo,
    name="Average State Income",
    data=state_average,
    columns=[state_average.index, "Average_State_Income"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Average State Income (present USD)",
).add_to(USmap2)


#folium.LayerControl().add_to(USmap2)

USmap2

In [ ]:
featuregroup4 = folium.FeatureGroup(name = "Average SBA-Backed Loan Value",
                                  overlay = False)

cp_income = folium.Choropleth(
    geo_data=state_geo,
    name="Average SBA-Backed Loan",
    data=grouped_by_state,
    columns=["State", "SBA_backed_portion"],
    key_on="feature.id",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Average Portion of Loan, SBA-Backed",
).add_to(USmap2)


folium.LayerControl().add_to(USmap2)

USmap2

In [156]:
USmap2.save("FINAL_income_SBA_loans_final.html")